In [4]:
%pip install torch torchvision
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [5]:
import torch, torchvision

print(torch.__version__)
print(torchvision.__version__)

2.1.2
0.16.2


In [6]:
import copy
import time

import numpy as np
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision import datasets, transforms, models

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
print(len(os.environ["CUDA_VISIBLE_DEVICES"].split(',')))  # prints 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


2


In [8]:
# drive.mount("/content/drive")
# root='/content/drive/MyDrive/Final Training Data'
# root='/content/drive/MyDrive/Vision/grape_dataset/test'

In [9]:
# Define transformations for the training, validation, and testing sets
training_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                 [0.229, 0.224, 0.225])])

# Load datasets

In [10]:
# Loading the datasets with ImageFolder
training_dataset = datasets.ImageFolder('Data/archive/grape_dataset/train', transform=training_transforms)
validation_dataset = datasets.ImageFolder('Data/archive/grape_dataset/test', transform=validation_transforms)

In [11]:
BATCH_SIZE = 64
# Using the image datasets and the transforms to define the dataloaders
train_loader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory = True)
valid_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available:
    print('lovin the GPU')

lovin the GPU


In [13]:
# Load the pretrained MobileNetV2 model from torchvision.models
model = models.mobilenet_v2(pretrained=True)

# Change the classifier part of the model
model.classifier[1] = nn.Linear(model.last_channel, 4)

# Define the criterion and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move the model to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

c:\Users\aless\miniconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aless\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

# Training the Model

# All torch models

In [14]:
from torchvision.models import densenet121, efficientnet_b7, mobilenet_v2
from torch.nn import Parameter


In [15]:
# Load the pretrained models from torchvision.models
models = [densenet121(pretrained=True), efficientnet_b7(pretrained=True), mobilenet_v2(pretrained=True)]
model_names = ['DenseNet121', 'EfficientNetB7', 'MobileNetV2']

c:\Users\aless\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\aless\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B7_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B7_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
import torch
from torchvision import datasets, transforms
from torchvision.models import densenet121

# Check if CUDA is available and set PyTorch to use GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the training set
training_transforms = transforms.Compose([transforms.RandomResizedCrop(224), 
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])


# Load the pretrained DenseNet121 model
model = densenet121(pretrained=True)
model = model.to(device)

# Function to build and train a model
def train_model(model):
    # Define the criterion and the optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training the model
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for epoch in range(25):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            print(f"Running loss: {running_loss:.3f}")

        print(f"Epoch {epoch+1}/{25}.. "
              f"Train loss: {running_loss/len(train_loader):.3f}.. ")

    accuracy = 100 * correct / total
    return running_loss, accuracy

# Train the model and print results
print("Training DenseNet121...")
loss, accuracy = train_model(model)
print(f"Results for DenseNet121: Loss = {loss:.3f}, Accuracy = {accuracy:.2f}%")


Training DenseNet121...
Running loss: 9.259
Running loss: 13.262
Running loss: 14.119


KeyboardInterrupt: 

In [ ]:
import torch
from torchvision import datasets, transforms
from torchvision.models import densenet121

# Define transformations for the test set
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

# Define a DataLoader for the test dataset
test_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=64)

# Load the pretrained DenseNet121 model
model = densenet121(pretrained=True)
model = model.to(device)

# Function to test the model
def test_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on test images: {100 * correct / total:.2f}%')

# Test the model
test_model(model)


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/path/to/your/test/dataset'